### Cloud GKE Storage Options<br>

***

<br>

##### **Docker Storage**<br>

docker volume is a directory on a disk, or in another container<br>
docker container has a writeable layer and it's where data is stored by default.<br>
data is not persisted, when the container is removed<br>
so storing data inside of container is not recommended<br>

there are 3 ways to mount the data inside of a docker container<br>
Pod(Container) -Volumes-> FileSystem(Docker Area)<br>

1. **Volumes** - sits inside docker area within the host's filesystem<br>
and can be shared amongst other containers<br>
this volume is a docker object and is decoupled from the container<br>
it can be attached and shared across multiple containers<br>

Pod(Container) -Bind Mount-> FileSystem<br>

2. **Bind Mount** - comes directly from host's file system<br>
bind mounts are great for local application development<br>
yet cannot be shared amongst other containers<br>

Pod(Container) -tmpfs-> Memory<br>

3. **tmpfs** - stored in host's memory<br>
increases performance as it no longer lies to container's writeable layer<br>
<br>

##### **Kubernetes Storage Abstractions**<br>

File systems and block based storage are provided to your pods<br>
but are different than docker in nature<br>

Ingress -> Service -> (ReplicaSet) Pod(s)<br>
**Volumes** - basic storage unit that decouples the storage from the container and tie it to the pod<br>
not the container like in docker<br>

Volume is a directory that the pods have access to, volume type used will determine it's purpose<br>
some volume types are backed by an ephemeral storage (Ex. empty dir, ConfigMap, Secret) and get removed with the pod<br>
it's useful for caching temporary information sharing files between containers or to load data into a pod<br>

**Persistent Volume/Persistent Volume Claim** - durable storage<br>
other volume types are backed by durable storage<br>
and persist beyond the lifetime of a pod<br>

Persistent Volume is a cluster resource that pods can use for a durable storage<br>
Persistent Volume Claim can be used to dynamically provision a persistent volume backed by persistent disks<br>
PVC can also be used to provision other types of backing storage like NFS<br>
<br>

**Volumes** - are the simplest way to write the data<br>
Files are lost if container crashes or stops for any other reason<br>
As well as being unaccessible to other containers running in the same pod<br>

in Kubernetes, the volume source declared in pod spec determines how the directory is created<br>
storage medium used and directory's initial contents<br>
pod specifies what volumes it contains and path where container mount the volume<br>

ephemeral volume types live the same amount of time as the pods they're connected to<br>
these volumes created when the pod is created and persists through container's restarts<br>
it gets terminated when pod is terminated or deleted<br>

##### **Types of Volumes**<br>

**emptyDir** - provides an empty directory that containers in the Pod can read/write from<br>
when the pod is removed from the node for any reason, data is removed forever<br>
emptyDir is stored in whatever medium backing the node, Disk/SSD/Network Storage<br>
useful for: scratch space, sharing data between multiple containers in a pod<br>

**ConfigMap** - resource that provides a way to inject configuration data into Pods<br>
data stored in configMap object can be referenced in a volume of a type configMap and<br>
then consumed through files running in a pod<br>

**Secret** - used to make sensitive data avilable to applications<br>
data stored in a secret object can be referenced in a volume of a type Secret and then<br>
consumed through files running in a pod<br>

**Downward API** - used to make Downward API data available to applications<br>
data includes information about the pod and container, in which an application is running it<br>
an Example of this would be to expose information about the pod's namespace and IP to applications<br>

**PersistentVolumeClaim** - provision durable storage to be used by applications<br>
a pod uses PVC to mount the volume that's backed by this durable storage<br>
<br>

##### **Persistent Volume**<br>

Persistent Volume resources are used to manage durable storage in a cluster<br>
in GKE, a persistent volume is typically backed by a persistent disk<br>
or filestore can be used as an NFS solution<br>
unlike volumes, persistent volume's lifecycle is managed by Kubernetes<br>
and can be dynamically provisioned without the need to manually create and delete the backing storage<br>
persistent volume resources are cluster resources that exists independently of pods<br>
and continue to exist as cluster changes, and pods are deleted and recreated<br>

Pod(s) <-> Persistent Volume Claim <-> Persistent Volume(backed by a disk)<br>
**Persistent Volume Claim** - this is a request for / claim to a persistent volume resource<br>
PVC requests a specific size, access mode and storage class for persistent volume<br>
Pods use Claims as volumes<br>
cluster inspects the claim to find the bind volume and mounts that volumes for the pod<br>


##### **Storage Classes**<br>

In [ ]:
apiVersion: storage.k8s.io/v1
kind: StorageClass # storageClass resources
metadata:
    name: standard
provisioner: kubernetes.io/gce-pd
parameters:
    type: pd-standard
    fstype: ext4 # default storage class
    replication-type: none
reclaimPolicy: Retain
allowVolumeExpansion: true
mountOptions:
    - debug
volumeBindingMode: Immediate

PVC can automatically provision persistent disk for you<br>
when you create PVC, Kubernetes dynamically creates a corresponding Persistent Volume Object<br>
due to the GKE default storage class, this persistent volume is backed by a new empty Compute Engine persistent disks<br>
you use this disks in a pod by using the claim as a volume<br>

when you delete a claim, the corresponding persistent disk object and provision compute engine persistent disks are also deleted<br>
To prevent deletion, you can **set Reclaim policy** to retain to prevent persistent volume deletion<br>

StatefulSet is recommended method of deploying Stateful applications that require a unique volume per replica<br>
by using StatefulSet with persistent volume claim templates, you can have apps that can scale up automatically with unique persistent volume claims<br>
associated with each replica pods<br>

##### **Persistent Volume Access**<br>

**Access Modes**<br>

- **ReadWriteOnce**<br>
where the volume can be mounted as read-write by a single node<br>
most common case for persistent disks, works as default access mode for most apps<br>

- **ReadOnlyMany**<br>
where the volume can be mounted as read-only by many nodes<br>

- **ReadWriteMany**<br>
where the volume can be mounted as read-write by many nodes<br>
<br>

**Type of persistent disks**<br>

- **Regional persistent disks**<br>
multi-zonal resources that replicate between 2 zones in the same region<br>
in case of outages, Kubernetes can failover workloads(HA) to the other zone<br>
high-available workloads for GKE<br>

- **Zonal persistent disks**<br>
zonal resources, if no zone is specified, it's chosen at random<br>
any pods referencing disk are scheduled in same zone as the disk<br>
Using anti-affinity on zones allows StatefulSet pods to be spread across zones along with the corresponding disks<br>
<br>

**Speed of access** - IOPS are determined by the size of the persistent disks<br>
use larger and fewer disks to achieve high IOPS and throughputs<br>

- **Cloud SQL (DATABASE)**<br>


- **Cloud Spanner (DATABASE)**<br>


- **Datastore (DATABASE)**<br>


- **Filestore (NAS(Network Attached Storage))**<br>


- **Cloud Storage (OBJECT STORAGE)**<br>


- **Persistent Disk (BLOCK STORAGE)**<br>

